In [77]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col,rank
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

spark=SparkSession.builder.appName("UDF – Standardize Currency Columns").getOrCreate()
df=spark.read.csv("/content/top_100_saas_companies_2025.csv",header=True,inferSchema=True)

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from pyspark.sql.types import DoubleType

spark=SparkSession.builder.appName("UDF – Standardize Currency Columns").getOrCreate()
df=spark.read.csv("/content/top_100_saas_companies_2025.csv",header=True,inferSchema=True)


def convert_to_values(k):
  k=k.split()
  k=str(k[0])
  m=1.0
  if k is None:
    return None
  if k=='N/A':
    return None
  if k[0]=="$":
    k=k[1:]
  if k.endswith("B"):
    k=k[:-1]
    m=m*1000000000

  elif k.endswith("M"):
    k=k[:-1]
    m=m*1000000
  elif k.endswith("T"):
    k=k[:-1]
    m=m*1000000000000
  elif k.endswith("K"):
    k=k[:-1]
    m=m*1000
  return float(k)*m



convert_to_values_udf=udf(convert_to_values,DoubleType())

df=df.withColumn("ARR_values",convert_to_values_udf(col("ARR")))
df=df.withColumn("Valuation_values",convert_to_values_udf(col("Valuation")))
df=df.withColumn("Total_Funding_values",convert_to_values_udf(col("Total Funding")))

df.select("Company Name", "Valuation", "Valuation_values",
                      "ARR", "ARR_values",
                      "Total Funding", "Total_Funding_values").show(20, truncate=False)


+------------+-------------------+----------------+------+--------------------+-------------+--------------------+
|Company Name|Valuation          |Valuation_values|ARR   |ARR_values          |Total Funding|Total_Funding_values|
+------------+-------------------+----------------+------+--------------------+-------------+--------------------+
|Microsoft   |$3T                |3.0E12          |$270B |2.7E11              |$1B          |1.0E9               |
|Salesforce  |$227.8B            |2.278E11        |$37.9B|3.79E10             |$65.4M       |6.540000000000001E7 |
|Adobe       |$240B              |2.4E11          |$19.4B|1.94E10             |$2.5M        |2500000.0           |
|Oracle      |$350B              |3.5E11          |$52.9B|5.29E10             |$2K          |2000.0              |
|SAP         |$215B              |2.15E11         |$32.5B|3.25E10             |N/A          |NULL                |
|Intuit      |$180B              |1.8E11          |$14.4B|1.44E10             |$

In [75]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col,rank
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

spark=SparkSession.builder.appName("UDF – Standardize Currency Columns").getOrCreate()
df=spark.read.csv("/content/top_100_saas_companies_2025.csv",header=True,inferSchema=True)


def convert_to_values(k):
  k=k.split()
  k=str(k[0])
  m=1.0
  if k is None:
    return None
  if k=='N/A':
    return None
  if k[0]=="$":
    k=k[1:]
  if k.endswith("B"):
    k=k[:-1]
    m=m*1000000000

  elif k.endswith("M"):
    k=k[:-1]
    m=m*1000000
  elif k.endswith("T"):
    k=k[:-1]
    m=m*1000000000000
  elif k.endswith("K"):
    k=k[:-1]
    m=m*1000
  return float(k)*m



convert_to_values_udf=udf(convert_to_values,DoubleType())
df=df.withColumn("valuation_values",convert_to_values_udf(col("Valuation")))


w=Window.partitionBy("industry").orderBy(col("valuation_values").desc())
df = df.withColumn("Rank", rank().over(w)) \
              .filter(col("Rank") <= 2)


df.select("Company Name", "Industry", "Valuation", "Valuation_values", "Rank") \
         .orderBy("Industry", "Rank") \
         .show(truncate=False)




+------------+----------------------+--------------+----------------+----+
|Company Name|Industry              |Valuation     |Valuation_values|Rank|
+------------+----------------------+--------------+----------------+----+
|AppDynamics |APM                   |$3.7B (Cisco) |3.7E9           |1   |
|Affirm      |BNPL                  |$12B          |1.2E10          |1   |
|Looker      |Business Intelligence |$2.6B (Google)|2.6E9           |1   |
|CircleCI    |CI/CD                 |$1.7B         |1.7E9           |1   |
|Salesforce  |CRM                   |$227.8B       |2.278E11        |1   |
|Marqeta     |Card Issuing          |$4.3B         |4.3E9           |1   |
|Zscaler     |Cloud Security        |$30B          |3.0E10          |1   |
|Netskope    |Cloud Security        |$7.5B         |7.5E9           |2   |
|Dropbox     |Cloud Storage         |$8.5B         |8.5E9           |1   |
|Box         |Cloud Storage         |$3.5B         |3.5E9           |2   |
|Miro        |Collaborati

In [80]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lag
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window


spark = SparkSession.builder.appName("ARR Growth Gaps").getOrCreate()

df = spark.read.csv("/content/top_100_saas_companies_2025.csv", header=True, inferSchema=True)


def parse_currency(k):
    if k is None or k == 'N/A':
        return None
    k = k.strip()
    multiplier = 1.0
    if k.startswith("$"):
        k = k[1:]
    if k.endswith("T"):
        k = k[:-1]
        multiplier = 1_000_000_000_000
    elif k.endswith("B"):
        k = k[:-1]
        multiplier = 1_000_000_000
    elif k.endswith("M"):
        k = k[:-1]
        multiplier = 1_000_000
    elif k.endswith("K"):
        k = k[:-1]
        multiplier = 1_000
    try:
        return float(k) * multiplier
    except:
        return None

parse_currency_udf = udf(parse_currency, DoubleType())

df = df.withColumn("ARR_Num", parse_currency_udf(col("ARR")))


w = Window.partitionBy("Industry").orderBy(col("ARR_Num").desc())


from pyspark.sql.functions import expr

df = df.withColumn("Prev_ARR", lag("ARR_Num").over(w))


df = df.withColumn("ARR_Drop", col("Prev_ARR") - col("ARR_Num"))


df_filtered = df.filter(col("ARR_Drop") > 1_000_000_000)


df_filtered.select("Company Name", "Industry", "ARR", "ARR_Num", "Prev_ARR", "ARR_Drop") \
           .orderBy("Industry", "ARR_Drop", ascending=[True, False]) \
           .show(truncate=False)


+------------+-------------------+------+-------+--------------------+--------------------+
|Company Name|Industry           |ARR   |ARR_Num|Prev_ARR            |ARR_Drop            |
+------------+-------------------+------+-------+--------------------+--------------------+
|Netskope    |Cloud Security     |$500M |5.0E8  |1.6E9               |1.1E9               |
|Box         |Cloud Storage      |$1B   |1.0E9  |2.5E9               |1.5E9               |
|RingCentral |Communications     |$2.2B |2.2E9  |4.0999999999999995E9|1.8999999999999995E9|
|CrowdStrike |Cybersecurity      |$3.1B |3.1E9  |7.5E9               |4.4E9               |
|Palantir    |Data Analytics     |$2.2B |2.2E9  |3.7E9               |1.5E9               |
|Redis       |Database           |$100M |1.0E8  |1.7E9               |1.6E9               |
|Figma       |Design             |$600M |6.0E8  |2.0E9               |1.4E9               |
|SAP         |Enterprise Software|$32.5B|3.25E10|2.7E11              |2.375E11  

In [82]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when


spark = SparkSession.builder.appName("G2 Rating Tier Classification").getOrCreate()


df = spark.read.csv("/content/top_100_saas_companies_2025.csv", header=True, inferSchema=True)


df = df.withColumn("Rating_Tier",
                   when(col("G2 Rating") >= 4.7, "Excellent")
                   .when((col("G2 Rating") >= 4.3) & (col("G2 Rating") < 4.7), "Very Good")
                   .when((col("G2 Rating") >= 4.0) & (col("G2 Rating") < 4.3), "Good")
                   .otherwise("Average"))


df.select("Company Name", "G2 Rating", "Rating_Tier") \
  .orderBy(col("G2 Rating").desc()) \
  .show(20, truncate=False)


+------------+---------+-----------+
|Company Name|G2 Rating|Rating_Tier|
+------------+---------+-----------+
|Rippling    |4.8      |Excellent  |
|Ramp        |4.8      |Excellent  |
|Miro        |4.7      |Excellent  |
|Notion      |4.7      |Excellent  |
|Figma       |4.7      |Excellent  |
|Verkada     |4.7      |Excellent  |
|Calendly    |4.7      |Excellent  |
|CrowdStrike |4.7      |Excellent  |
|Gong        |4.7      |Excellent  |
|LaunchDarkly|4.7      |Excellent  |
|Cloudflare  |4.7      |Excellent  |
|Canva       |4.7      |Excellent  |
|Navan       |4.7      |Excellent  |
|Grammarly   |4.6      |Very Good  |
|HashiCorp   |4.6      |Very Good  |
|Monday.com  |4.6      |Very Good  |
|Airtable    |4.6      |Very Good  |
|Brex        |4.6      |Very Good  |
|Klaviyo     |4.6      |Very Good  |
|Benchling   |4.6      |Very Good  |
+------------+---------+-----------+
only showing top 20 rows



In [83]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, trim


spark = SparkSession.builder.appName("Investor Tier Enrichment").getOrCreate()


df = spark.read.csv("/content/top_100_saas_companies_2025.csv", header=True, inferSchema=True)


investor_tiers = spark.createDataFrame([
    ("Accel", "Tier 1"),
    ("Sequoia", "Tier 1"),
    ("Andreessen Horowitz", "Tier 1"),
    ("SoftBank", "Tier 2"),
    ("Lightspeed", "Tier 2"),
    ("Unknown", "Tier 3")
], ["Investor", "Tier"])


df = df.withColumn("First_Investor", trim(split(col("Top Investors"), ",")[0]))


joined_df = df.join(investor_tiers, df["First_Investor"] == investor_tiers["Investor"], "inner")


filtered_df = joined_df.filter(col("Tier").isin("Tier 1", "Tier 2"))


result_df = filtered_df.select("Company Name", "First_Investor", "Tier", "Valuation")


result_df.orderBy("Tier", "Company Name").show(truncate=False)


+------------------+-------------------+------+--------------------+
|Company Name      |First_Investor     |Tier  |Valuation           |
+------------------+-------------------+------+--------------------+
|Algolia           |Accel              |Tier 1|$2.3B               |
|Amplitude         |Sequoia            |Tier 1|$4B                 |
|Canva             |Sequoia            |Tier 1|$40B                |
|Carta             |Andreessen Horowitz|Tier 1|$7.4B               |
|Confluent         |Sequoia            |Tier 1|$9.1B               |
|Databricks        |Andreessen Horowitz|Tier 1|$43B                |
|Dropbox           |Sequoia            |Tier 1|$8.5B               |
|Freshworks        |Accel              |Tier 1|$5.2B               |
|Gong              |Sequoia            |Tier 1|$7.3B               |
|Mixpanel          |Andreessen Horowitz|Tier 1|$1.05B              |
|MongoDB           |Sequoia            |Tier 1|$26B                |
|Navan             |Andreessen Hor

In [85]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, when
from pyspark.sql.types import DoubleType

# Step 1: Start Spark session
spark = SparkSession.builder.appName("Valuation vs Industry Median").getOrCreate()

# Step 2: Load SaaS company data
df = spark.read.csv("/content/top_100_saas_companies_2025.csv", header=True, inferSchema=True)

# Step 3: Define currency parser UDF
def parse_currency(k):
    if k is None or k == 'N/A':
        return None
    k = k.strip()
    multiplier = 1.0
    if k.startswith("$"):
        k = k[1:]
    if k.endswith("T"):
        k = k[:-1]
        multiplier = 1_000_000_000_000
    elif k.endswith("B"):
        k = k[:-1]
        multiplier = 1_000_000_000
    elif k.endswith("M"):
        k = k[:-1]
        multiplier = 1_000_000
    elif k.endswith("K"):
        k = k[:-1]
        multiplier = 1_000
    try:
        return float(k) * multiplier
    except:
        return None

parse_currency_udf = udf(parse_currency, DoubleType())

df = df.withColumn("Valuation_values", parse_currency_udf(col("Valuation")))


industry_medians = spark.createDataFrame([
    ("Enterprise Software", 150_000_000_000),
    ("CRM", 100_000_000_000),
    ("AI", 70_000_000_000),
    ("HRTech", 50_000_000_000),
], ["Industry", "Median_Valuation"])


joined_df = df.join(industry_medians, on="Industry", how="inner")


joined_df = joined_df.withColumn(
    "Valuation_Position",
    when(col("Valuation_values") > col("Median_Valuation"), "Above Median")
    .otherwise("Below Median")
)


joined_df.select("Company Name", "Industry", "Valuation", "Valuation_values", "Median_Valuation", "Valuation_Position") \
         .orderBy("Industry", "Valuation_values", ascending=[True, False]) \
         .show(truncate=False)


+------------+-------------------+---------+----------------+----------------+------------------+
|Company Name|Industry           |Valuation|Valuation_values|Median_Valuation|Valuation_Position|
+------------+-------------------+---------+----------------+----------------+------------------+
|Salesforce  |CRM                |$227.8B  |2.278E11        |100000000000    |Above Median      |
|Microsoft   |Enterprise Software|$3T      |3.0E12          |150000000000    |Above Median      |
|SAP         |Enterprise Software|$215B    |2.15E11         |150000000000    |Above Median      |
+------------+-------------------+---------+----------------+----------------+------------------+

